In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
import xgboost

In [3]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [18]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')
analizar = pd.read_csv('../properati_dataset_modificado2.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat','lon','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']

In [20]:
datos = propiedades.loc[:,columnas]
resultados = propiedades.loc[:, 'price_aprox_usd']

set_entrenamiento_datos,set_pruebas_datos, set_entrenamiento_resultado, set_pruebas_resultado = \
train_test_split(datos, resultados, test_size = 0.1, random_state = 42)

In [ ]:
criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 20
arbol = DecisionTreeRegressor()

inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

set_pruebas_datos.loc[:,'resultado_Decision_Tree'] = set_pruebas_datos.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)
analizar.loc[:,'resultado_Decision_Tree'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

In [ ]:
lista_k = [1,2,3,10,15,20,30,50]
lista_dist = [1,2,5]

parametros = {"n_neighbors" : lista_k, "p" : lista_dist}

iteraciones_cross_validation = 5
knn = KNeighborsRegressor()

inicio = time.strftime("%X")

grid = GridSearchCV( estimator = knn, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

set_pruebas_datos.loc[:,'resultado_KNN'] = set_pruebas_datos.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)
analizar.loc[:,'resultado_KNN'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

In [ ]:
estimators = [1000]
samples = [1.0]
features = [1.0]

parametros = {"n_estimators" : estimators, "max_samples" : samples, "max_features" : features}

iteraciones_cross_validation = 3
bag = BaggingRegressor()

inicio = time.strftime("%X")

grid = GridSearchCV( estimator = bag, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

set_pruebas_datos.loc[:,'resultado_Bagging_Regressor'] = set_pruebas_datos.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)
analizar.loc[:,'resultado_Bagging_Regressor'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

In [ ]:
joblib.dump(grid, 'algoritmos/Bagging_Regressor_blending.pkl')

In [ ]:
# ALgoritmo predictor final
columnas = columnas + ['resultado_Decision_Tree', 'resultado_KNN', 'resultado_Bagging_Regressor']

criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 10
arbol = DecisionTreeRegressor()

inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_pruebas_datos.loc[:,columnas], set_pruebas_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

In [ ]:
joblib.dump(grid, 'algoritmos/Blending_final.pkl')

In [ ]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Ensamble_con_Blending.csv', index = False)
#956216016563.16200